In [ ]:
import sqlite3
from datetime import datetime, timedelta

In [ ]:
#Connect to local database (create it if it is not existing)
conn = sqlite3.connect('office_data.db')
cursor = conn.cursor()

In [ ]:
#cursor.execute("drop table attendance")

In [ ]:
#three columns data table - id - primary(no two samples can have same id), name , date

table_query = """
CREATE TABLE attendance(id integer primary key autoincrement, name text, date text)
"""

cursor.execute(table_query)

In [ ]:
print(datetime.now())

2025-08-19 16:20:01.661339


In [ ]:
datetime.now().strftime("%Y-%m-%d")

'2025-08-19'

In [ ]:
#insert few data samples
#"insert into attendance (name, date) values (?, ?)"

today = datetime.now()

for i in range(3):
  day = today - timedelta(days = i)
  for j in range(100 + i * 10):
    cursor.execute("insert into attendance (name, date) values (?, ?)" ,(f"person_{j+1}" , day.strftime("%Y-%m-%d")))

conn.commit()
conn.close()

In [ ]:
conn = sqlite3.connect('office_data.db')
cursor = conn.cursor()

cursor.execute("select * from attendance limit 5")
rows = cursor.fetchall()
for row in rows:
  print(row)

conn.commit()
conn.close()

(1, 'person_1', '2025-08-19')
(2, 'person_2', '2025-08-19')
(3, 'person_3', '2025-08-19')
(4, 'person_4', '2025-08-19')
(5, 'person_5', '2025-08-19')


In [ ]:
conn = sqlite3.connect('office_data.db')
cursor = conn.cursor()

count_query = "select count(*) from attendance where date = '2025-08-17'"
cursor.execute(count_query)
count = cursor.fetchone()
print(count)

conn.commit()
conn.close()

(120,)


In [ ]:
def get_attendance_count(date_str):
    conn = sqlite3.connect('office_data.db')
    cursor = conn.cursor()

    cursor.execute("select count(*) from attendance where date = ?", (date_str,))
    count = cursor.fetchone()

    conn.commit()
    conn.close()

    return count

In [ ]:
get_attendance_count('2025-08-18')[0]

110

In [ ]:
def handle_query(user_query):
  if 'yesterday' in user_query.lower():
    date = (datetime.now() - timedelta(days = 1)).strftime("%Y-%m-%d")
    count = get_attendance_count(date)
    return f"Bot: Yesterday, {count} people were in the office."
  else:
    return "Bot: I can only answer about yesterday count as of now."

In [ ]:
user_query = "How many people were in the office yesterday?"
response = handle_query(user_query)
print(response)

Bot: Yesterday, (110,) people were in the office.


**Bot that can answer attendance for any day**

In [ ]:
user_query = "How many people were in the office on 2025-08-18"

import re

print(re.search(r"\d{4}-\d{2}-\d{2}" , user_query))
print(re.search(r"\d{4}-\d{2}-\d{2}" , user_query).group())

<re.Match object; span=(38, 48), match='2025-08-18'>
2025-08-18


In [ ]:
#Function to extract date from the query
def extract_date_from_query(user_query):
  today = datetime.now().date()
  if "today" in user_query.lower():
    return today.strftime("%Y-%m-%d")
  elif "yesterday" in user_query.lower():
    yesterday = today - timedelta(days = 1)
    return yesterday.strftime("%Y-%m-%d")
  else:
    #Match the given date
    date_match = re.search(r"\d{4}-\d{2}-\d{2}" , user_query)
    if date_match:
      return date_match.group()
    return None

In [ ]:
extract_date_from_query("How many people are there in office today")

'2025-08-19'

In [ ]:
extract_date_from_query("How many people were there in office yesterday")

'2025-08-18'

In [ ]:
extract_date_from_query("How many people were there in office on '2025-08-17'")

'2025-08-17'

In [ ]:
#Chabot query handler
def chatbot_query_handler(user_query):
  target_date = extract_date_from_query(user_query)
  if not target_date:
    return """Bot: Sorry I couldn't find a date in your query,
    Please try again with a valid date or use words like today or yesterday"""

  count = get_attendance_count(target_date)[0]

  if not count:
    return f"Bot: No attendance record for the {target_date}"
  else:
    return f"Bot: on {target_date} , {count} people were in the office"

In [ ]:
querries = [
    "How many people were in the office today?",
    "How many people were in the office on 2025-08-17?",
    "How many people were in the office yesterday?",
    "How many people were in the office on 2025-08-16?",
    "How many people were in the office on 2025-08-20?",
    "What is attendance?"
]

for query in querries:
  print(f"User: {query}")
  response = chatbot_query_handler(query)
  print(response)
  print()

User: How many people were in the office today?
Bot: on 2025-08-19 , 100 people were in the office

User: How many people were in the office on 2025-08-17?
Bot: on 2025-08-17 , 120 people were in the office

User: How many people were in the office yesterday?
Bot: on 2025-08-18 , 110 people were in the office

User: How many people were in the office on 2025-08-16?
Bot: No attendance record for the 2025-08-16

User: How many people were in the office on 2025-08-20?
Bot: No attendance record for the 2025-08-20

User: What is attendance?
Bot: Sorry I couldn't find a date in your query,
    Please try again with a valid date or use words like today or yesterday



In [ ]:
user_input = input("What do you want to know?")

response = chatbot_query_handler(user_input)
print(response)

What do you want to know?2025-08-18
Bot: on 2025-08-18 , 110 people were in the office
